# codecentric.AI Bootcamp &mdash; NLP / Übungsaufgaben

*Hallo und herzlich willkommen zum codecentric.AI bootcamp!*

Im Tutorial haben wir bereits eine Anwendung von NLP vorgestellt. Nun wollen wir Dich in einem
praktischen Teil dazu einladen, selbst ein paar typische NLP-Techniken auszuprobieren und zu implementieren. Dabei erfährst Du,

1. wie elegant sich N-Gramme in Python extrahieren lassen,
2. wieviel Stopp-Wörter Politiker verwenden,
3. wie man in wenigen Zeilen "Wer fällt aus der Reihe?" programmiert,
4. was sich hinter dem tf-idf-Maß verbirgt und wie man scikit-learn-Transformer verwendet.

Damit es gleich losgehen kann, benötigen wir die folgenden Bibliotheken.

*Viel Spaß!*

In [1]:
import os
import numpy as np
import pandas as pd
import gensim

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()

## Aufgabe 1: Von einzelnen Wörtern zu Wortgruppen

Als erste Möglichkeit, einen Text zu analysieren, haben wir uns dessen *einzelne Wörter* beziehungsweise dessen bag of words angeschaut. Oft ist es hilfreich, zusätzlich *Wortgruppen* anzuschauen, etwa alle auftretenden Wortpaare. Für den Beispielsatz

> "Fischers Fritz fischt frische Fische"

wären das also

> "Fischers Fritz", "Fritz fischt", "fischt frische", "frische Fische".

Allgemein bezeichnet man als [N-Gramm](https://de.wikipedia.org/wiki/N-Gramm) ein Tupel von N aufeinander folgenden Token. Ist N=2 wie im Beispiel, so spricht man von _Bigrammen_.

### (a)  Bigramme extrahieren

Schreibe eine Funktion `bigrams`, die eine Liste `words` von Wörtern als Eingabe nimmt und die Menge aller auftretenden Bigramme zurückgibt. Tipp: Verwende `zip`!

In [55]:
def bigrams(words):
    return set(zip(words[:-1], words[1:]))

In [56]:
WORDS = ["Fischers", "Fritz", "fischt", "frische", "Fische"]
bigrams(WORDS)


{('Fischers', 'Fritz'),
 ('Fritz', 'fischt'),
 ('fischt', 'frische'),
 ('frische', 'Fische')}

### (b) N-Gramme extrahieren

Schreibe nun eine Funktion `ngrams`, die eine Liste `words` von Wörtern und eine Zahl `n` erwartet und die Menge aller n-Gramme zurückgibt. Tipp: mit `zip(*iters)` kann man `zip` auf eine ganze Liste `iters` von `Iterable`s anwenden. 

In [95]:
def ngrams(words, n):
    l = len(words) - n + 1
    return set(zip(*[words[i:l + i] for i in range(0,n)]))

In [96]:
ngrams(WORDS,3)

{('Fischers', 'Fritz', 'fischt'),
 ('Fritz', 'fischt', 'frische'),
 ('fischt', 'frische', 'Fische')}

## Aufgabe 2: Wer verwendet die meisten Stopp-Wörter?

Ein üblicher Arbeitsschritt bei NLP ist die Entfernung sogenannter [Stopp-Wörter](https://de.wikipedia.org/wiki/Stoppwort), die häufig auftreten, aber für den Anwendungsfall keine wesentliche Informationen beinhalten. Wir testen, wieviel Stopp-Wörter in den Reden auftauchen, die wir im Tutorial klassifiziert hatten. Dazu lesen wir die Reden nochmal ein:

In [ ]:
DATA_PATH = "data"
ANALYSIS_FILE = "speeches.pickle"
ANALYSIS_PATH = os.path.join(DATA_PATH, ANALYSIS_FILE)

df = pd.read_pickle(ANALYSIS_PATH)

### (a) Stopp-Wörter filtern 

Eine Liste von 231 Stopp-Wörtern ist in [NLTK](https://www.nltk.org/) enthalten. Wir haben diese Stopp-Wörter zeilenweise in der Datei `data/stopwords_nltk.txt` abgespeichert. Schreibe

- eine Funktion `read_stopwords`, die die Liste der Stopp-Wörter einliest und als Menge zurückgibt,
- eine Funktion  `filter_words`, die aus einer Liste `words` alle Wörter ausfiltert, die in `stopwords` enthalten sind, und die bereinigte Liste zurückgibt. 

Tips:

- Öffne die Datei der Stopwörter mit der Option `encoding="utf-8"`.
- Benutze gegebenenfalls die Methode [rstrip](https://docs.python.org/3/library/stdtypes.html), um Zeilenumbrüche von eingelesenen Strings zu entfernen.
- `filter_words` sollte ein Einzeiler sein.

Teste Deine Funktionen wie folgt:

In [ ]:
EXAMPLE = ["Das", "ist", "ein", "ganz", "und", "gar", "normaler", "Satz", "mit", "vielen", "Wörtern"]
stopwords = read_stopwords()
filter_words(EXAMPLE, stopwords)

### (b) Anwendung auf die Reden

Füge nun dem pandas-DataFrame `df` eine Spalte `filtered_tokens` hinzu, die für jede Rede die Liste der Token enthält, aus denen die Stopp-Wörter entfernt wurden.


Füge als Nächstes `df` eine Spalte `stop_percentage` hinzu, die für jede Rede angibt, wieviel Prozent der Rede Stopp-Wörter waren.

### (d) Stopp-Wort-Anteile visualisieren

Plotte abschließend den prozentualen Anteil an Stopp-Wörtern in den Redern nach Politikern gruppiert in einem geeigneten. Wodurch fallen Kanzler auf?

Tipp: Verwende [kategorielle Plot-Funktionen](https://seaborn.pydata.org/tutorial/categorical.html) von seaborn oder schau im Tutorial nach.

## Aufgabe 3: Wer fällt aus der Reihe?

Word embeddings kann man nicht nur für ernsthaftes NLP verwenden, sondern auch, um Spaß zu haben. Im Tutorial hatten wir schon ein kleines Tabu-Spiel programmiert. Nun wollen wir in wenigen Zeilen eine Funktion `find_the_odd` schreiben, die aus einer Liste von Wörtern den Ausreißer herausfindet &mdash; das Wort, das aus der Reihe fällt. Zum Beispiel also

> Fruehling, Sommer, Abend, Herbst
> => Abend


Dazu benötigen wir wieder die Wortvektoren aus der [Arbeit](https://devmount.github.io/GermanWordEmbeddings/) von [Andreas Müller](https://github.com/devmount).

In [ ]:
WV_PATH = os.path.join("data", "german.model.reduced")

w2v = gensim.models.KeyedVectors.load(WV_PATH)

### (a) Ähnlichkeitsmatrix berechnen

Die Wortvektoren sind nun in als gensim-`KeyedVectors`-Objekt `w2v` verfügbar. Die Methode `similarity` berechnet zu gegebenen Wörtern nun deren [Cosinus-Ähnlichkeit](https://de.wikipedia.org/wiki/Kosinus-%C3%84hnlichkeit): 0 heißt keine Ähnlichkeit, 1 heißt Übereinstimmung.

In [ ]:
w2v.similarity("Mensch", "Maschine")

Schreibe nun eine Funktion `similarity_matrix`, die eine Liste `words` von Wörtern als Eingabe nimmt und ein 2-dimensionales numpy-Array zurückliefert, dessen Eintrag an der Stelle `[i,j]` die Cosinus-Ähnlichkeit zwischen den Wörtern `words[i]` und `words[j]` ist. 

In [ ]:
def similarity_matrix(words):
    pass

Tipp: Mit Hilfe der numpy-Funktion [asarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.asarray.html) und List-Komprehensionen geht das in zwei Zeilen!

### (b) Ausreißer finden 

Um den Ausreißer in einer Liste `words` von Wörtern zu finden, können wir nun die Matrix `similarity_matrix(words)` nehmen und die Einträge jeder Zeile aufsummieren: die Zeile mit der kleinsten Summe entspricht dem Ausreißer. Schreibe eine Funktion `find_the_odd`, welche dies umsetzt. Hilfreich sind dabei die numpy-Funktionen [np.sum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html) und [np.argmin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmin.html).

In [ ]:
def find_the_odd(words):
    pass

### (c) Probier es aus! 

Teste nun Deine Funktion an folgenden Beispielen und denk Dir selber welche aus! Beachte dabei, dass alle Wörter groß geschrieben und die Umlaute `ä`, `ö`, `ü` sowie `ß` als `ae`, `oe`, `ue` beziehungsweise `ss`umgeschrieben werden müssen. 

In [ ]:
ODDS = [["Fruehling", "Sommer", "Abend", "Herbst"],
        ["Fruehstueck", "Oma", "Abendbrot", "Mittagessen"],
        ["Rot", "Gruen", "Blau", "Maler"],
        ["Hund", "Mensch", "Hase", "Katze"],
        ["Tochter", "Mutter", "Grossmutter", "Cousine"]
    ]

for odd_set in ODDS:
    print(", ".join(odd_set), " => ", find_the_odd(odd_set))

## Aufgabe 4: Klassifikation mit dem tf-idf-Maß und scikit-learn

In dem Tutorial  haben wir zwei statistische Größen für die Klassifikation der Reden genutzt: *welche* Token, Grundformen beziehungsweise Begriffe auftauchen, zusammengefasst in einer *bag of words*, und *wie oft* jedes Token et cetera in jeder Rede auftaucht. In dieser Aufgabe lernen wir weitere wichtige Größen kennen:

- die *Vorkommenshäufigkeit*, welche im Englischen *term frequency* (*tf*) genannt wird,
- die *inverse Dokumentvorkommenshäufigkeit*, englisch *inverse document frequency* (*idf*),
- das kombinierte [tf-idf-Maß](https://de.wikipedia.org/wiki/Tf-idf-Ma%C3%9F).

Diese messen jeweils

- wie "wichtig" das Token für die Rede ist &mdash; je häufiger, desto wichtiger,
- wie "spezifisch" das Token für die gesamte Redensammlung ist &mdash; je häufiger, desto unwichtiger,
- eine Kombination beider Aspekte: ein Token ist umso bedeutender, je öfter es in der Rede auftaucht und je seltener in anderen.

Mit Hilfe dieser Größen wollen wir nun die Reden, die wir bereits im Tutorial angeschaut haben, noch einmal klassifizieren. Dazu laden wir erstmal den aufbereiteten Datensatz.

In [ ]:
DATA_PATH = "data"
ANALYSIS_FILE = "speeches.pickle"

df = pd.read_pickle(os.path.join(DATA_PATH, ANALYSIS_FILE))

### (a) Vorverarbeitung mit scikit-learn-Transformern

[scikit-learn](https://scikit-learn.org) bietet zahlreiche Hilfsmittel für die Vorverarbeitung von Daten, unter anderem die Klasse [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Bestimme damit die Tfidf-Statistiken der Reden in folgenden Schritten:

1. Bestimme die Menge aller Token, die in den Reden auftreten, und nennen diese Menge `vocab`;
2. erzeuge mit `TfIdfVectorizer(vocabulary=vocab)` ein Objekt `vectorizer`;
3. bilde für jede Rede aus der Liste der Token eine Zeichenkette, indem die Token mit Leerzeichen verknüpft werden;
4. wende die Methode `vectorizer.fit_transform` auf das erhaltene `Iterable` von Zeichenketten an und nenne das Ergebnis `tfidf`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


Wenn alles geklappt hat, können wir uns die wichtigsten Terme einer Rede wie folgt anschauen:

In [ ]:
REDE = 13 # Index der Rede
termsAndTfidfs = list(zip(vocab, tfidf.toarray()[REDE]))
termsAndTfidfs.sort(reverse=True, key=lambda termAndTfidf: termAndTfidf[1])
termsAndTfidfs[:25]

### (b) Klassifikation mit dem tf-idf-Maß

Nun kannst Du mit den gewonnenen tf-idf-Daten einen Bayes-Klassifizierer trainieren. Die wesentlichen Schritte sind wie im Tutorial

1. Zerlegung der Daten in Trainings- und Testdaten &mdash; dafür bietet scikit-learn die Routine [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);
2. das eigentliche Training mit Hilfe von [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) und der Methode `fit` sowie
3. die Beurteilung des Trainings-Erfolgs mit Hilfe der Testdaten mit Hilfe der Methode `predict` und der Funktion [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB


Es stellt sich wahrscheinlich heraus, dass keine zufriedenstellende Qualität erreicht wird. Natürlich könnte oder sollte man das wie im Tutorial durch wiederholte Tests genauer prüfen. Der Grund ist schnell gefunden: pro Redner haben wir im Schnitt _zu wenig Reden_.